## 데이터 로드

In [1]:
import json
import pandas as pd

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [2]:
train_data = pd.DataFrame(json_data)
train_data = train_data.drop(['id', 'plylst_title', 'updt_date', 'like_cnt'], axis=1)
train_data.head()

,tags,songs
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [3]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [4]:
song_data = pd.DataFrame(json_data)
song_data = song_data.drop(['album_name', 'song_gn_gnr_basket'], axis=1)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 열 이름 변경

In [5]:
train_data.rename(columns={'songs':'song_id'}, inplace=True)
train_data.head()

,tags,song_id
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [6]:
song_data.rename(columns={'id':'song_id', 'song_gn_dtl_gnr_basket': 'gnr'}, inplace=True)
song_data = song_data.astype({'issue_date':'int64'})
song_data.head()

,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,song_id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 추출

- 500개의 플레이리스트 추출

In [7]:
train_data_sample = train_data[:500]

## 태그 병합

- 같은 노래에 부여된 서로 다른 태그들을 합친다
- 그 결과 동일한 태그 리스트가 거의 모든 노래에 부여되었다

In [8]:
train_data_sample = train_data_sample.explode('song_id', ignore_index=True)
train_data_sample.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [9]:
train_dict = dict()

for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'][i]
    tag = train_data_sample['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)
        
print(train_dict[157435])

{'kpop', '스트레스해소', '여자아이돌', '걸그룹댄스', '댄스'}


In [10]:
train_data_sample.drop_duplicates(subset='song_id', keep='first',inplace=True)
train_data_sample.shape

(16674, 2)

In [11]:
for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'].iloc[i]
    
    train_data_sample['tags'][i] = list(train_dict[song])

train_data_sample.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [12]:
merge = pd.merge(train_data_sample, song_data)
merge.head()

,tags,song_id,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket
0,[락],525514,"[GN1402, GN1401]",20130506,2200223,[734201],Hey Little Girl,[The Sol]
1,[락],129701,"[GN0901, GN0902, GN1001]",20130917,2201802,[536907],Octagon,[Royal Bangs]
2,[락],383374,"[GN1012, GN1005, GN1001]",19911021,2216938,[166978],The Road,[Honeymoon Suite]
3,[락],562083,"[GN1013, GN0901, GN0902, GN1001]",20000919,43227,[19035],Honeymoon,[Phoenix]
4,[락],297861,"[GN1013, GN0901, GN0902, GN1001]",20050306,303657,[170117],High,[James Blunt]


## Word2Vec 사용

- 태그 리스트들을 word2vec로 학습시켜 태그 하나와 연관된 다른 태그들을 유추

In [13]:
train_data_sample2 = train_data[:500]

In [14]:
from gensim.models.word2vec import Word2Vec

w2v = Word2Vec(sentences = train_data_sample2['tags'], vector_size = 100, 
               window = 5, min_count = 5, workers = 4, sg = 0)

w2v.wv.vectors.shape

(66, 100)

In [15]:
print(w2v.wv.most_similar('스트레스'))

[('분위기', 0.22454456984996796), ('힙합', 0.17852607369422913), ('국힙', 0.16757138073444366), ('이별', 0.1608145833015442), ('팝', 0.15153270959854126), ('휴식', 0.13615800440311432), ('여름', 0.13451433181762695), ('설렘', 0.11924614757299423), ('추억', 0.11664089560508728), ('회상', 0.11426529288291931)]


## 코사인 유사도 사용

- 세부 장르를 사용해 코사인 유사도 측정한다
- 그후 유사도를 행렬로 저장한다

In [16]:
train_data_explode = train_data_sample2.explode('song_id', ignore_index=True)
train_data_explode.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [17]:
train_data_explode.drop_duplicates(['song_id'], ignore_index=True)
train_data_explode.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [18]:
merge2 = pd.merge(train_data_explode['song_id'], song_data)
merge2.head()

,song_id,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket
0,525514,"[GN1402, GN1401]",20130506,2200223,[734201],Hey Little Girl,[The Sol]
1,129701,"[GN0901, GN0902, GN1001]",20130917,2201802,[536907],Octagon,[Royal Bangs]
2,383374,"[GN1012, GN1005, GN1001]",19911021,2216938,[166978],The Road,[Honeymoon Suite]
3,562083,"[GN1013, GN0901, GN0902, GN1001]",20000919,43227,[19035],Honeymoon,[Phoenix]
4,297861,"[GN1013, GN0901, GN0902, GN1001]",20050306,303657,[170117],High,[James Blunt]


In [19]:
from sklearn.feature_extraction.text import CountVectorizer

merge2['gnr_literal'] = merge2['gnr'].apply(lambda x : (' ').join(x))

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
gnr_mat = count_vect.fit_transform(merge2['gnr_literal'])

gnr_mat.shape

(21425, 835)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

gnr_sim = cosine_similarity(gnr_mat, gnr_mat)
gnr_sim[0]

array([1., 0., 0., ..., 0., 0., 0.])

- 노래 id가 주어지면 유사도 순으로 n개의 노래 추출

In [21]:
import numpy as np

def find_sim_song(df, sim_matrix, songs, top_n=10):
    simi = np.zeros(len(df['song_id']))
    maxyear = 0
    minyear = 3000
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        maxyear = max(maxyear, title_song['issue_date'].values[0]//10000)
        minyear = min(minyear, title_song['issue_date'].values[0]//10000)
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        title_index = title_song.index.values
    
        simi = simi + sim_matrix[title_index, :]
    
    simi /= len(songs)
    
    df['similarity'] = simi.reshape(-1, 1)
    temp = df.sort_values(by="similarity", ascending=False)
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        title_index = title_song.index.values
        
        temp = temp[temp.index.values != title_index]
    
    temp = temp[temp['issue_date'] > minyear*10000]
    temp = temp[temp['issue_date'] < (maxyear+1)*10000]
    
    final_index = temp.index.values[ : top_n]
    
    return df.iloc[final_index]

In [22]:
similar_songs = find_sim_song(merge2, gnr_sim, [525514, 129701, 229622], 10)
similar_songs[['song_id', 'similarity', 'issue_date']]

,song_id,similarity,issue_date
13358,598147,0.551551,20131105
13353,233718,0.551551,20130409
7787,9336,0.551551,20110909
7781,693335,0.551551,20121015
7780,237693,0.551551,20121015
7779,682132,0.551551,20121015
7778,33428,0.551551,20121015
7777,443713,0.551551,20111115
1291,310974,0.551551,20061107
1290,310974,0.551551,20061107


## 노래 추천

- w2v로 추출한 태그에 해당하는 플레이리스트
- 세부 장르의 유사도가 높은 노래 리스트
- 히스토리(test 플레이리스트)의 발행 연도와 같은 연도에 발행한 노래

In [23]:
def song_recommend(tags, songs, tag_df, song_df, sim_mat):
    ts = tags
    
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 5개까지 늘린다
    while len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            for i in range(5):
                stag = w2v.wv.most_similar(tag)[i][0]
                
                if not stag in ts:
                    ts.append(stag)
                
                if len(ts) >= 3:
                    break
            if len(ts) >= 3:
                break
    
    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                
                for ss in tag_df['song_id'][i]:
                    if not ss in songs:
                        
                        if ss in tag_songs:
                            tag_songs[ss] += 1
                            
                        else:
                            tag_songs[ss] = 1
                        
    tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        simi_songs = find_sim_song(song_df, sim_mat, songs, 100)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        for song in songs:
            title_song = song_df[song_df['song_id'] == song]
            title_index = title_song.index.values
        
            simi_songs = simi_songs[simi_songs.index.values != title_index]
    
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    index = 0
    
    while len(recommended) < 10 and index < len(tag_songs):
        tag_song = tag_songs[index][0]
        
        if tag_song in simi_songs:
            recommended.append(tag_song)
            
        index += 1
        
    # 둘 모두에 존재하는 노래가 10개 미만인 경우
    # 각각에서 우선순위가 높은 노래들을 추출한다   
    if len(recommended) < 10:
        if len(recommended) % 2 == 0:
            sc = (10-len(recommended)) / 2
        else:
            sc = (10-len(recommended)) // 2
        
        index = 0
        while len(tag_songs) != 0 and len(recommended) < (10 - sc):
            tag_song = tag_songs[index][0]
            
            if not tag_song in recommended:
                recommended.append(tag_song)
            
            index += 1
        
        index = 0
        while len(recommended) < 10:
            simi_song = simi_songs['song_id'].values[index]
            
            if not simi_song in recommended:
                recommended.append(simi_song)
            
            index += 1
            
    # 추출된 노래 id를 가지고 데이터프레임을 추출한다
    rec_index = []
    
    for rec in recommended:
        title_song = song_df[song_df['song_id'] == rec]
        title_index = title_song.index
        rec_index.append(title_index[0]) # 중복제거를 했음에도 중복이 제거 안됨
    
    return song_df.loc[rec_index]

### 태그와 노래 목록 존재

In [24]:
my_tags1 = ['락']
my_songs1 = [525514, 129701, 229622]
rec1 = song_recommend(my_tags1, my_songs1, train_data_sample2, merge2, gnr_sim)
rec1[['song_id', 'song_name', 'artist_name_basket', 'issue_date']]

,song_id,song_name,artist_name_basket,issue_date
1295,146989,YOUTH,[Troye Sivan],20160123
1303,430106,Everglow,[Coldplay],20151204
1342,15124,Something Just Like This,"[The Chainsmokers, Coldplay]",20170407
3514,258814,Surrender (Feat. 린),[챈슬러 (Chancellor)],20161130
3520,86380,오늘도 그대만 (Feat. 정동원),[T.P RETRO (타디스 프로젝트舊)],20170228
13358,598147,Take Your Time,[Louis Yoelin],20131105
13353,233718,Another Day Alone,[Louis Yoelin],20130409
7787,9336,Lego House,[Ed Sheeran],20110909
7781,693335,Broken,[Jake Bugg],20121015
7780,237693,Taste It,[Jake Bugg],20121015


### 노래 목록만 존재

In [25]:
my_tags2 = []
my_songs2 = [525514, 129701, 229622]
rec2 = song_recommend(my_tags2, my_songs2, train_data_sample2, merge2, gnr_sim)
rec2[['song_id', 'song_name', 'artist_name_basket', 'issue_date']]

,song_id,song_name,artist_name_basket,issue_date
13358,598147,Take Your Time,[Louis Yoelin],20131105
13353,233718,Another Day Alone,[Louis Yoelin],20130409
7787,9336,Lego House,[Ed Sheeran],20110909
7781,693335,Broken,[Jake Bugg],20121015
7780,237693,Taste It,[Jake Bugg],20121015
7779,682132,Two Fingers,[Jake Bugg],20121015
7778,33428,Lightning Bolt,[Jake Bugg],20121015
7777,443713,Give Me Love,[Ed Sheeran],20111115
1287,310974,9 Crimes,[Damien Rice],20061107
3421,322130,Mrs. Cold,[Kings Of Convenience],20090924


### 태그만 존재

In [26]:
my_tags3 = ['락']
my_songs3 = []
rec3 = song_recommend(my_tags3, my_songs3, train_data_sample2, merge2, gnr_sim)
rec3[['song_id', 'song_name', 'artist_name_basket', 'issue_date']]

,song_id,song_name,artist_name_basket,issue_date
1295,146989,YOUTH,[Troye Sivan],20160123
1303,430106,Everglow,[Coldplay],20151204
1342,15124,Something Just Like This,"[The Chainsmokers, Coldplay]",20170407
3514,258814,Surrender (Feat. 린),[챈슬러 (Chancellor)],20161130
3520,86380,오늘도 그대만 (Feat. 정동원),[T.P RETRO (타디스 프로젝트舊)],20170228
125,394031,Into the Unknown (From &#34;Frozen 2&#34;/Soun...,"[Idina Menzel, Aurora]",20191115
183,457519,꿀차,[우효],20180102
184,453762,너 정말 예쁘다,[최낙타],20180410
185,349398,LOVE YA!,[혁오 (HYUKOH)],20180531
186,631142,편지,[장희원],20180603


### 둘 다 없음
- 태그가 없는 경우에도 w2v을 통해 태그가 생성됨

In [27]:
my_tags4 = []
my_songs4 = []
rec4 = song_recommend(my_tags4, my_songs4, train_data_sample2, merge2, gnr_sim)
rec4[['song_id', 'song_name', 'artist_name_basket', 'issue_date']]

,song_id,song_name,artist_name_basket,issue_date
125,394031,Into the Unknown (From &#34;Frozen 2&#34;/Soun...,"[Idina Menzel, Aurora]",20191115
183,457519,꿀차,[우효],20180102
184,453762,너 정말 예쁘다,[최낙타],20180410
185,349398,LOVE YA!,[혁오 (HYUKOH)],20180531
186,631142,편지,[장희원],20180603
188,406082,하늘엔 별이 떠있고 너만큼은 빛나질 않아,[이민혁],20180903
189,548389,사랑에 빠졌네,[정준일],20181101
190,205179,꿀맛,[정미애],20190815
194,567076,숨겨진 세상 (Into the Unknown End Credit Version) (...,[태연 (TAEYEON)],20191115
286,418694,Attention,[Charlie Puth],20180511


## 추천 시스템 평가 방법
- Reference(1): https://yeomko.tistory.com/32
- Reference(2): https://chrisjune-13837.medium.com/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C-%EC%84%B1%EB%8A%A5%ED%8F%89%EA%B0%80%EB%B0%A9%EB%B2%95-with-python-9932097f0ff9
- 모델 성능 vs 사용자 주관적 평가
  - 사실 추천 시스템의 성능 지표로는 후자가 더 정확하지만, 개인의 주관은 정량적 측정이 불가능
  - A/B 테스트라도 해보려면 일단 성능이 기본적으로 받쳐줘야 한다.
- Ranking 기반 추천 시스템 평가 지표
  - mAP
  - nDCG
  - Entropy Diversity

### mAP(Mean Average Precision)
- 사용자가 좋아한 노래의 리스트와 시스템이 추천한 리스트를 비교
- 플레이리스트의 절반을 숨기고 나머지를 맞추도록 한 후 비교 평가하는 방식 구현

In [28]:
def ap_at_k(like_item, recommend_item, k):
    precisions = []
    # 1부터 K까지 Loop를 돌며 Precision을 계산합니다
    for i in range(k):
        # 아래로직은 Precision과 동일합니다
        base = list(recommend_item)[:i+1]
        intersect = set(base).intersection(like_item)
        relevance = recommend_item[i] in like_item
        print("intersect:",intersect, "base:", base, "relevance", relevance)

        precisions.append(len(intersect) / len(base) * relevance)
    print("precisions: ", precisions)

    ap_k = sum(precisions) / len(precisions)
    print("ap@k: ", ap_k)
    return ap_k

In [29]:
# 임의의 10곡짜리 플레이리스트 생성
my_playlist_map = [146989, 430106, 15124, 258814, 86380, 394031, 457519, 453762, 349398, 631142]
my_tags_map = ['락']
my_playlist_map_test = [146989, 430106, 15124, 258814, 86380]

my_rec_map = song_recommend(my_tags_map, my_playlist_map_test, train_data_sample2, merge2, gnr_sim)
my_rec_map[['song_id', 'song_name', 'artist_name_basket', 'issue_date']]

ValueError: operands could not be broadcast together with shapes (8,21425) (6,21425) 

In [ ]:
ap_at_k(my_playlist_map, my_rec_map['song_id'], 10)

### nDCG(Normalized Discounted Cumulative Gain)
- 추천 순서를 고려해 가중치를 매기는 방법(상위에 있을수록 중요도 높음)
- CG: 각 아이템에 대해 부여한 점수(relevance score)의 총합
- DCG: 각 relevance score에 순서를 고려해 log2(i + 1)로 나누어 먼저 위치한 아이템일수록 우선순위가 높아지도록 함
- nDCG: DCG / iDCG의 값
  - iDCG: relevance score 벡터가 내림차순으로 되어있을 경우에 나타나는 가장 이상적인 DCG 값(= 추천이 가장 잘 된 경우)
  - 1에 가까울수록 잘 추천한 것

### 적용 방법
- 위의 mAP 계산 때 사용한 플레이리스트를 재사용, 똑같이 절반을 가리고 실험 예정
- 플레이리스트에서 DCG는 Precision 기준 True를 1, False를 0으로 계산하는 relevance score를 활용

In [ ]:
my_playlist_ndcg = [146989, 430106, 15124, 258814, 86380, 394031, 457519, 453762, 349398, 631142]
my_tags_ndcg = ['락']
my_playlist_ndcg_test = [146989, 430106, 15124, 258814, 86380]

my_rec_ndcg = song_recommend(my_tags_ndcg, my_playlist_ndcg_test, train_data_sample2, merge2, gnr_sim)
my_rec_ndcg[['song_id', 'song_name', 'artist_name_basket', 'issue_date']]

In [ ]:
# 여기에서 추천 플리와 실제 플리를 비교하는 nDCG 계산 코드를 쓸 예정...인데 아직 mAP 오류 해결 중

### Entropy Diversity
- 추천 음악의 다양성을 측정하는 방법
- 분류 문제에서의 Entropy 개념을 확장한 것
  - 장르가 다양할수록 Entropy 증가, 반대로 장르가 유사할수록 Entropy 감소
- 너무 다양함과 너무 유사함 사이에서 중심을 잡아야 할 듯
- 코드 구현은 아직...